In [6]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import joblib
import os
import glob
import warnings
warnings.filterwarnings('ignore')

#DD-MM-YYYY,store_nbr,item_nbr,units
#29-07-2012  40	5	48
#31-07-2012	23	2	0
#31-07-2012	23	3	0
#31-07-2012	23	4	0
#31-07-2012	23	5	25

#def predict():
#setting up input
passbackInput = ['2014-10-31', '44', '68']
dateVal  = passbackInput[0]     
storeVal = int(passbackInput[1])  
itemVal  = int(passbackInput[2])

#************************ Classification model **************
#------------------------------------------------------------
# Reading CSV files for Classification model
path = os.getcwd()
df_cls = pd.read_csv(os.path.join(path, "df_Classifier_cleand.csv"))
#------------------------------------------------------------
#Check if data already exsist in CSV or not
df_chkRecord = df_cls.loc[(df_cls['date'] == dateVal) & (df_cls['store_nbr'] == storeVal) & (df_cls['item_nbr'] == itemVal)]
#------------------------------------------------------------
print(len(df_chkRecord))
ActualValue = "New data point- No previous value"
CheckAvailable = "NOT AVAILABLE"
if len(df_chkRecord)==0:
    df_weatherVal1 = df_cls.loc[df_cls['Month']==int(passbackInput[0].split("-")[1])] 
    df_weatherVal2 = df_weatherVal1.loc[df_weatherVal1['store_nbr'] == int(storeVal)]
    df_weatherVal_median = df_weatherVal2.median()
    
    QueryPoint_Dummy = df_weatherVal_median.to_frame()
    columnLists = ["store_nbr","item_nbr","station_nbr","tmax","depart","cool","snowfall","preciptotal","stnpressure","resultspeed","resultdir","avgspeed","Day",
    "Month","Holiday","codesum_BCFG","codesum_BLDU","codesum_BLSN","codesum_BR","codesum_DU","codesum_DZ","codesum_FG","codesum_FG+","codesum_FU",
    "codesum_FZDZ","codesum_FZFG","codesum_FZRA","codesum_GR","codesum_GS","codesum_HZ","codesum_MIFG","codesum_PL","codesum_PRFG","codesum_RA",
    "codesum_SG","codesum_SN","codesum_SQ","codesum_TS","codesum_TSRA","codesum_TSSN","codesum_UP","codesum_VCFG","codesum_VCTS","codesum_nan","units"]
    values = [] 
    for clmn in columnLists:
        values.append(QueryPoint_Dummy[0][clmn])

    QueryPoint = pd.DataFrame([values],columns = columnLists)
    QueryPoint['item_nbr'] = itemVal
else:
    QueryPoint = df_chkRecord
    ActualValue = str(df_chkRecord['units'])#str(df_chkRecord.get_value(0, 'units'))
    CheckAvailable = "AVAILABLE"


if 'date' in QueryPoint.columns:
    QueryPoint.drop(['date'],axis = 1,inplace=True)

if 'flag' in QueryPoint.columns:
    QueryPoint.drop(['flag'],axis = 1,inplace=True)    

#If data point exsists
if len(df_chkRecord)==1:
    lr = joblib.load('cls_LogisticRegrsn.pkl')
    prediction = lr.predict(QueryPoint)
    print(prediction[0])

if ((prediction[0] == 1 & len(df_chkRecord)==1) or (len(df_chkRecord)==0)):
    if 'units' in QueryPoint.columns:
        QueryPoint.drop(['units'],axis = 1,inplace=True)
   
    #Standardize the Query point
    #for clmn in QueryPoint.columns:
        # fit on data column & Transform
        #scale = StandardScaler().fit(QueryPoint[[clmn]])
        #QueryPoint[clmn] = scale.transform(QueryPoint[[clmn]])  
        
    # Our best model is XGBRegressor, then Random Forest. 
    #But for both these Model- while loading pkl file we faced error in terms of Parameter so used Linear Regression  
    reg_RF = joblib.load('LinearReg.pkl')
    predictionReg = reg_RF.predict(QueryPoint)
    FinalPrediction = abs(predictionReg)
    
elif (prediction[0] == 0) & (len(df_chkRecord)==1):
    FinalPrediction = abs(prediction[0])
    
FinalPrediction


1
1


array([647.42486592])

In [5]:
df_cls[df_cls['units']>0].tail(20)

,date,store_nbr,item_nbr,units,station_nbr,tmax,depart,cool,snowfall,preciptotal,...,codesum_TSRA,codesum_TSSN,codesum_UP,codesum_VCFG,codesum_VCTS,codesum_nan,Day,Month,Holiday,flag
4611769,2014-10-31,38,45,63,14,60.0,-7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,31,10,False,1
4611844,2014-10-31,39,9,12,8,78.0,1.5,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4611871,2014-10-31,39,36,3,8,78.0,1.5,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4611938,2014-10-31,39,103,4,8,78.0,1.5,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4611946,2014-10-31,39,111,2,8,78.0,1.5,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4611951,2014-10-31,40,5,18,13,58.0,1.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,31,10,False,1
4611991,2014-10-31,40,45,44,13,58.0,1.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,31,10,False,1
4612073,2014-10-31,41,16,30,12,75.0,1.5,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4612125,2014-10-31,41,68,15,12,75.0,1.5,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1
4612165,2014-10-31,41,108,7,12,75.0,1.5,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,31,10,False,1


In [7]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import joblib
import os
import glob
import warnings
warnings.filterwarnings('ignore')

#DD-MM-YYYY,store_nbr,item_nbr,units
#29-07-2012  40	5	48
#31-07-2012	23	2	0
#31-07-2012	23	3	0
#31-07-2012	23	4	0
#31-07-2012	23	5	25

#def predict():
#setting up input
passbackInput = ['2014-10-31', '44', '68']
dateVal  = passbackInput[0]     
storeVal = int(passbackInput[1])  
itemVal  = int(passbackInput[2])

#************************ Classification model **************
#------------------------------------------------------------
# Reading CSV files for Classification model
path = os.getcwd()
df_cls = pd.read_csv(os.path.join(path, "df_Classifier_cleand.csv"))
#------------------------------------------------------------
#Check if data already exsist in CSV or not
df_chkRecord = df_cls.loc[(df_cls['date'] == dateVal) & (df_cls['store_nbr'] == storeVal) & (df_cls['item_nbr'] == itemVal)]
#------------------------------------------------------------
print(len(df_chkRecord))
ActualValue = "New data point- No previous value"
CheckAvailable = "NOT AVAILABLE"
if len(df_chkRecord)==0:
    df_weatherVal1 = df_cls.loc[df_cls['Month']==int(passbackInput[0].split("-")[1])] 
    df_weatherVal2 = df_weatherVal1.loc[df_weatherVal1['store_nbr'] == int(storeVal)]
    df_weatherVal_median = df_weatherVal2.median()
    
    QueryPoint_Dummy = df_weatherVal_median.to_frame()
    columnLists = ["store_nbr","item_nbr","station_nbr","tmax","depart","cool","snowfall","preciptotal","stnpressure","resultspeed","resultdir","avgspeed","Day",
    "Month","Holiday","codesum_BCFG","codesum_BLDU","codesum_BLSN","codesum_BR","codesum_DU","codesum_DZ","codesum_FG","codesum_FG+","codesum_FU",
    "codesum_FZDZ","codesum_FZFG","codesum_FZRA","codesum_GR","codesum_GS","codesum_HZ","codesum_MIFG","codesum_PL","codesum_PRFG","codesum_RA",
    "codesum_SG","codesum_SN","codesum_SQ","codesum_TS","codesum_TSRA","codesum_TSSN","codesum_UP","codesum_VCFG","codesum_VCTS","codesum_nan","units"]
    values = [] 
    for clmn in columnLists:
        values.append(QueryPoint_Dummy[0][clmn])

    QueryPoint = pd.DataFrame([values],columns = columnLists)
    QueryPoint['item_nbr'] = itemVal
else:
    QueryPoint = df_chkRecord
    ActualValue = str(df_chkRecord['units'])#str(df_chkRecord.get_value(0, 'units'))
    CheckAvailable = "AVAILABLE"


if 'date' in QueryPoint.columns:
    QueryPoint.drop(['date'],axis = 1,inplace=True)

if 'flag' in QueryPoint.columns:
    QueryPoint.drop(['flag'],axis = 1,inplace=True)    

#If data point exsists
if len(df_chkRecord)==1:
    lr = joblib.load('cls_LogisticRegrsn.pkl')
    prediction = lr.predict(QueryPoint)
    print(prediction[0])

if ((prediction[0] == 1 & len(df_chkRecord)==1) or (len(df_chkRecord)==0)):
    if 'units' in QueryPoint.columns:
        QueryPoint.drop(['units'],axis = 1,inplace=True)
   
    #Standardize the Query point
    #for clmn in QueryPoint.columns:
        # fit on data column & Transform
        #scale = StandardScaler().fit(QueryPoint[[clmn]])
        #QueryPoint[clmn] = scale.transform(QueryPoint[[clmn]])  
        
    # Our best model is XGBRegressor, then Random Forest. 
    #But for both these Model- while loading pkl file we faced error in terms of Parameter so used Linear Regression  
    reg_RF = joblib.load('regrsn_XGBRegressor.pkl')
    predictionReg = reg_RF.predict(QueryPoint)
    FinalPrediction = abs(predictionReg)
    
elif (prediction[0] == 0) & (len(df_chkRecord)==1):
    FinalPrediction = abs(prediction[0])
    
FinalPrediction

1
1


XGBoostError: [19:31:34] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:959: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.



In [8]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import joblib
import os
import glob
import warnings
warnings.filterwarnings('ignore')

#DD-MM-YYYY,store_nbr,item_nbr,units
#29-07-2012  40	5	48
#31-07-2012	23	2	0
#31-07-2012	23	3	0
#31-07-2012	23	4	0
#31-07-2012	23	5	25

#def predict():
#setting up input
passbackInput = ['2014-10-31', '44', '68']
dateVal  = passbackInput[0]     
storeVal = int(passbackInput[1])  
itemVal  = int(passbackInput[2])

#************************ Classification model **************
#------------------------------------------------------------
# Reading CSV files for Classification model
path = os.getcwd()
df_cls = pd.read_csv(os.path.join(path, "df_Classifier_cleand.csv"))
#------------------------------------------------------------
#Check if data already exsist in CSV or not
df_chkRecord = df_cls.loc[(df_cls['date'] == dateVal) & (df_cls['store_nbr'] == storeVal) & (df_cls['item_nbr'] == itemVal)]
#------------------------------------------------------------
print(len(df_chkRecord))
ActualValue = "New data point- No previous value"
CheckAvailable = "NOT AVAILABLE"
if len(df_chkRecord)==0:
    df_weatherVal1 = df_cls.loc[df_cls['Month']==int(passbackInput[0].split("-")[1])] 
    df_weatherVal2 = df_weatherVal1.loc[df_weatherVal1['store_nbr'] == int(storeVal)]
    df_weatherVal_median = df_weatherVal2.median()
    
    QueryPoint_Dummy = df_weatherVal_median.to_frame()
    columnLists = ["store_nbr","item_nbr","station_nbr","tmax","depart","cool","snowfall","preciptotal","stnpressure","resultspeed","resultdir","avgspeed","Day",
    "Month","Holiday","codesum_BCFG","codesum_BLDU","codesum_BLSN","codesum_BR","codesum_DU","codesum_DZ","codesum_FG","codesum_FG+","codesum_FU",
    "codesum_FZDZ","codesum_FZFG","codesum_FZRA","codesum_GR","codesum_GS","codesum_HZ","codesum_MIFG","codesum_PL","codesum_PRFG","codesum_RA",
    "codesum_SG","codesum_SN","codesum_SQ","codesum_TS","codesum_TSRA","codesum_TSSN","codesum_UP","codesum_VCFG","codesum_VCTS","codesum_nan","units"]
    values = [] 
    for clmn in columnLists:
        values.append(QueryPoint_Dummy[0][clmn])

    QueryPoint = pd.DataFrame([values],columns = columnLists)
    QueryPoint['item_nbr'] = itemVal
else:
    QueryPoint = df_chkRecord
    ActualValue = str(df_chkRecord['units'])#str(df_chkRecord.get_value(0, 'units'))
    CheckAvailable = "AVAILABLE"


if 'date' in QueryPoint.columns:
    QueryPoint.drop(['date'],axis = 1,inplace=True)

if 'flag' in QueryPoint.columns:
    QueryPoint.drop(['flag'],axis = 1,inplace=True)    

#If data point exsists
if len(df_chkRecord)==1:
    lr = joblib.load('cls_LogisticRegrsn.pkl')
    prediction = lr.predict(QueryPoint)
    print(prediction[0])

if ((prediction[0] == 1 & len(df_chkRecord)==1) or (len(df_chkRecord)==0)):
    if 'units' in QueryPoint.columns:
        QueryPoint.drop(['units'],axis = 1,inplace=True)
   
    #Standardize the Query point
    #for clmn in QueryPoint.columns:
        # fit on data column & Transform
        #scale = StandardScaler().fit(QueryPoint[[clmn]])
        #QueryPoint[clmn] = scale.transform(QueryPoint[[clmn]])  
        
    # Our best model is XGBRegressor, then Random Forest. 
    #But for both these Model- while loading pkl file we faced error in terms of Parameter so used Linear Regression  
    reg_RF = joblib.load('RandomForestRegressor.pkl')
    predictionReg = reg_RF.predict(QueryPoint)
    FinalPrediction = abs(predictionReg)
    
elif (prediction[0] == 0) & (len(df_chkRecord)==1):
    FinalPrediction = abs(prediction[0])
    
FinalPrediction

1
1


AttributeError: 'DecisionTreeRegressor' object has no attribute 'n_features_'